In [1]:
#Importing Pandas
import pandas as pd

#Reading CSV files
Checkins = pd.read_csv("check_ins.csv")
Sessions = pd.read_csv("sessions.csv")

In [2]:
# Data manipulation

# Field names changed to be more descriptive
Checkins.rename(columns = {'Kind':'Session Type','Location':'Community'}, inplace = True)
Sessions.rename(columns = {'Kind':'Session Type','Location':'Community'}, inplace = True)

#Adding Kind field as "Drop-In Tutoring" to describe all data records
Checkins['Session Type'] = 'Drop-In Tutoring'

# Adding Month and Year data fields to extract specific date data
Sessions['Started_At_Year'] = pd.DatetimeIndex(Sessions['Started At Date']).year
Sessions['Started_At_Month'] = pd.DatetimeIndex(Sessions['Started At Date']).month
Checkins['Started_At_Year'] = pd.DatetimeIndex(Checkins['Started At Date']).year
Checkins['Started_At_Month'] = pd.DatetimeIndex(Checkins['Started At Date']).month

In [3]:
# Creating Union between sessions.csv and check_ins.csv

#Creating smaller tables with extracted date parts
Checkins_Date_df = Checkins[["Check In At Date","Community","Status", "Session Type", "Course", "Student Email","Student Name","Started At Date", "Started At Time", "Ended At Date", "Ended At Time", "Duration Minutes","Started_At_Year","Started_At_Month"]]
Sessions_Date_df = Sessions[["Requested At Date","Community","Status", "Session Type", "Course", "Student Email","Student Name","Started At Date", "Started At Time", "Ended At Date", "Ended At Time", "Tutor Submitted Length","Started_At_Year","Started_At_Month"]]

#Defining tables to preform concat function
Date_df1 = Checkins_Date_df[["Community","Status", "Session Type", "Course", "Student Email","Student Name","Started At Date", "Started At Time", "Ended At Date", "Ended At Time", "Started_At_Year","Started_At_Month"]]
Date_df2 = Sessions_Date_df[["Community","Status", "Session Type", "Course", "Student Email","Student Name","Started At Date", "Started At Time", "Ended At Date", "Ended At Time", "Started_At_Year","Started_At_Month"]]

#Union between Date_df1 and Date_df2
All_Dates_df = pd.concat([Date_df1,Date_df2], ignore_index = True) 

In [4]:
# First df - Top 10 count of total Tutoting sessions by community 

#Calculating count of tutoring sessions by Community
All_Cnt_Com = All_Dates_df.groupby(["Community"])["Community"].count().reset_index(name="Total Tutoring Sessions").sort_values(by='Total Tutoring Sessions', ascending=False).reset_index(drop=True)

#Calculating percent of total for Session Count by Community
All_Cnt_Com['Session_Pct'] = ((All_Cnt_Com['Total Tutoring Sessions']/All_Cnt_Com['Total Tutoring Sessions'].sum())*100).round(0)

#Formatting All_Cnt_Com
All_Cnt_Com = All_Cnt_Com.head(10).style.set_properties(**{'text-align': 'left'}).set_table_styles(
[dict(selector = 'th', props=[('text-align', 'left')])])

#Top 10 Communities by count of tutoring sessions W/ % of total
All_Cnt_Com

,Community,Total Tutoring Sessions,Session_Pct
0,Zoom Meeting - Online,419,33.000000
1,Student Success Center - In Person,305,24.000000
2,Academic Support Center,108,8.000000
3,Library 3rd Floor - In Person,79,6.000000
4,Zoom Meeting Online,58,5.000000
5,Student Success Center,54,4.000000
6,Library - Third Floor,35,3.000000
7,Student Success Center - Main Campus,32,3.000000
8,CSM 201 - Math Commons,30,2.000000
9,Math Lab - In Person,17,1.000000


In [5]:
# Second df - count of total Tutoting sessions by session type

#Calculating count of tutoring sessions by Session Type
All_Cnt_Typ = All_Dates_df.groupby(["Session Type"])["Session Type"].count().reset_index(name="Total Tutoring Sessions").sort_values(by='Total Tutoring Sessions', ascending=False).reset_index(drop=True)

#Calculating percent of total for Session Count by Session Type
All_Cnt_Typ['Session_Pct'] = ((All_Cnt_Typ['Total Tutoring Sessions']/All_Cnt_Typ['Total Tutoring Sessions'].sum())*100).round(0)

#Formatting All_Cnt_Typ
All_Cnt_Typ = All_Cnt_Typ.head(10).style.set_properties(**{'text-align': 'left'}).set_table_styles(
[dict(selector = 'th', props=[('text-align', 'left')])])

#Total tutoring sessions (for both datasets) by Session Type (Kind) W/ % of total
All_Cnt_Typ

,Session Type,Total Tutoring Sessions,Session_Pct
0,Group,599,47.000000
1,Drop-In Tutoring,388,30.000000
2,1-on-1,292,23.000000


In [6]:
# Third df - Top 10 Count of drop-in sessions by community and status W/ % of total

#Calculating tutoring session Count grouped by Community Partittion on Status 
df_Cm_Stat = All_Dates_df.groupby(["Community","Status"])["Community"].count().reset_index(name="Total Tutoring sessions").sort_values(by='Total Tutoring sessions', ascending=False).reset_index(drop=True)

#Calculating percent of total for Session Count by Community Partittion on Status 
df_Cm_Stat['Session_Pct'] = ((df_Cm_Stat['Total Tutoring sessions']/df_Cm_Stat['Total Tutoring sessions'].sum())*100).round(0)

#Formatting df_LS
All_Cm_Sat = df_Cm_Stat.head(10).style.set_properties(**{'text-align': 'left'}).set_table_styles(
[dict(selector = 'th', props=[('text-align', 'left')])])

#Top 10 Count of drop-in sessions by community and status W/ % of total
All_Cm_Sat

,Community,Status,Total Tutoring sessions,Session_Pct
0,Zoom Meeting - Online,Completed,265,21.000000
1,Student Success Center - In Person,Completed,211,16.000000
2,Zoom Meeting - Online,Cancelled,140,11.000000
3,Academic Support Center,Check In,108,8.000000
4,Student Success Center - In Person,Cancelled,83,6.000000
5,Student Success Center,Check In,54,4.000000
6,Library 3rd Floor - In Person,Completed,44,3.000000
7,Library 3rd Floor - In Person,Cancelled,35,3.000000
8,Student Success Center - Main Campus,Check In,32,3.000000
9,Zoom Meeting Online,Completed,31,2.000000


In [8]:
# Fourth df - Top 10 Count of drop-in sessions by community, session type, and status W/ % of total

#Calculating tutoring session Count grouped by Community Partittion on Session Type and Status 
All_Typ_Stat =  All_Dates_df.groupby(["Community","Session Type","Status"])["Community"].count().reset_index(name="Total Tutoring sessions").sort_values(by='Total Tutoring sessions', ascending=False).reset_index(drop=True)

#Calculating percent of total for Session Count by Community Partittion on Session Type and Status 
All_Typ_Stat['Session_Pct'] = ((All_Typ_Stat['Total Tutoring sessions']/All_Typ_Stat['Total Tutoring sessions'].sum())*100).round(0)

#Formatting df_LS
Cnt_Typ_stat = All_Typ_Stat.head(10).style.set_properties(**{'text-align': 'left'}).set_table_styles(
[dict(selector = 'th', props=[('text-align', 'left')])])

#Top 10 Count of drop-in sessions by community, session type, and status W/ % of total
Cnt_Typ_stat 

,Community,Session Type,Status,Total Tutoring sessions,Session_Pct
0,Zoom Meeting - Online,Group,Completed,234,18.000000
1,Student Success Center - In Person,Group,Completed,192,15.000000
2,Academic Support Center,Drop-In Tutoring,Check In,108,8.000000
3,Zoom Meeting - Online,1-on-1,Cancelled,73,6.000000
4,Zoom Meeting - Online,Group,Cancelled,67,5.000000
5,Student Success Center,Drop-In Tutoring,Check In,54,4.000000
6,Student Success Center - In Person,Group,Cancelled,48,4.000000
7,Student Success Center - In Person,1-on-1,Cancelled,35,3.000000
8,Student Success Center - Main Campus,Drop-In Tutoring,Check In,32,3.000000
9,Zoom Meeting - Online,1-on-1,Completed,31,2.000000


In [9]:
#Fifth df - Total count of completed tutoring sessions  grouped by status, year, and month W/ % of total

# Count of total tutoring sesions grouped by status, month, and year
df_Mth_Cnt = All_Dates_df.groupby(["Started_At_Year","Started_At_Month","Status"])["Status"].count().reset_index(name="Total Appointments").sort_values(['Started_At_Year','Started_At_Month']).reset_index(drop=True).round({'Started_At_Year': 0, 'Started_At_Month': 0})

#Formatting df_Typ_St filtering Count of total tutoring sesions by "completed" status
df_Mth_Cnt_Cmplt = df_Mth_Cnt[df_Mth_Cnt.Status == "Completed"].reset_index(drop=True).round({'Started_At_Year':0, 'Started_At_Month': 0}).style.set_properties(**{'text-align': 'left'}).set_table_styles(
[dict(selector = 'th', props=[('text-align', 'left')])])

#Total count of completed tutoring sessions grouped by date and status W/ % of total
df_Mth_Cnt_Cmplt

,Started_At_Year,Started_At_Month,Status,Total Appointments
0,2021.000000,6.000000,Completed,2
1,2021.000000,7.000000,Completed,12
2,2021.000000,8.000000,Completed,56
3,2021.000000,9.000000,Completed,87
4,2021.000000,10.000000,Completed,136
5,2021.000000,11.000000,Completed,92
6,2021.000000,12.000000,Completed,22
7,2022.000000,1.000000,Completed,8
8,2022.000000,2.000000,Completed,36
9,2022.000000,3.000000,Completed,53


In [10]:
#Sixth df - Total count of Check-in tutoring sessions grouped by status, year, and month W/ % of total

#Formatting df_Typ_St and filtering Count of total tutoring sesions by "Check in" status
df_Mth_Cnt_Chk = df_Mth_Cnt[df_Mth_Cnt.Status == "Check In"].reset_index(drop=True).round({'Started_At_Year':0, 'Started_At_Month': 0}).style.set_properties(**{'text-align': 'left'}).set_table_styles(
[dict(selector = 'th', props=[('text-align', 'left')])])

df_Mth_Cnt_Chk

,Started_At_Year,Started_At_Month,Status,Total Appointments
0,2021.000000,6.000000,Check In,15
1,2021.000000,7.000000,Check In,13
2,2021.000000,8.000000,Check In,13
3,2021.000000,9.000000,Check In,13
4,2021.000000,10.000000,Check In,13
5,2021.000000,11.000000,Check In,27
6,2021.000000,12.000000,Check In,15
7,2022.000000,1.000000,Check In,17
8,2022.000000,2.000000,Check In,19
9,2022.000000,3.000000,Check In,16
